In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')

In [2]:
!pip install pyspark==2.4.5

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 217.8 MB 47.6 MB/s eta 0:00:01                      | 20.6 MB 10.4 MB/s eta 0:00:19     |███████████████▌                | 105.3 MB 24.8 MB/s eta 0:00:05     |█████████████████████▊          | 147.8 MB 43.7 MB/s eta 0:00:02
     |████████████████████████████████| 197 kB 38.9 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=fa93136f6029c1f3dae22eb02f5117da6fdd77e51eb0db6c8113e76b60d9840f
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/01/c

In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [4]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In [5]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

--2021-05-07 18:11:00--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2021-05-07 18:11:00--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2021-05-07 18:11:00--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93299

In [6]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")

In [7]:
# this is the first time we have imported from the clustering package
# and not the classification package
from pyspark.ml.clustering import KMeans

# Here, our knowledge of the subject matter tells us that the data
# has 13 different movement patterns, so we set the number of clusters to be 13
# using the '.setK()' method when constructing the KMeans object

kmeans = KMeans().setK(13).setSeed(1)

# now notice that in constructing the KMeans object, we do not have to
# specify the column that KMeans works on.  Is this because it knows
# to look for "features"?  The pipeline is executed and works on the dataframe
# called 'df' below.

In [8]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, kmeans])

In [9]:
df.createOrReplaceTempView('df')
df = spark.sql("select * from df where class in ('Brush_teeth','Climb_stairs')")

In [10]:
# no need to divide the data into training and test
model = pipeline.fit(df)

In [11]:
# "within set sum of squared errors" is a measure where you compute the distance of 
# each point to its nearest cluster centroid and square it
# and the dataframe does not contain the feature vector, so 
# we have to call vectorAssember and pass it to the computeCost function

wssse = model.stages[1].computeCost(vectorAssembler.transform(df))
print("Within Set Sum of Squared Errors = " + str(wssse))

Within Set Sum of Squared Errors = 2667976.4624330676


In [12]:
# there is room for improvement.  But actually this notebook doesn't really make sense, because we
# set up the model for 13 clusters and then we gave it something with 2 classes

In [13]:
kmeans = KMeans().setK(2).setSeed(1)

In [14]:
# no need to divide the data into training and test
model = pipeline.fit(df)

In [15]:
wssse = model.stages[1].computeCost(vectorAssembler.transform(df))
print("Within Set Sum of Squared Errors = " + str(wssse))

Within Set Sum of Squared Errors = 2667976.4624330676


In [16]:
# ok so this is done, but not clear what the idea was here, except to show the code to use K-Means